In [233]:
%reset

In [234]:
# we are trying to predict the all nba team selections for the 2024 season
import pandas as pd

In [235]:
stats = pd.read_csv("player_all_nba_stats.csv")
stats_2024 = pd.read_csv("player_2024_stats.csv")

In [236]:
del stats["Unnamed: 0"] #unneeded column
del stats_2024["Unnamed: 0"]

In [237]:
stats = stats.fillna(0) # i am making it so that all values that are na, are being filled in with 0
stats_2024 = stats_2024.fillna(0)

In [238]:
# There is a new rule in which a player must play at least 65 games in order to be eligible for an all nba team recognition. https://www.cbssports.com/nba/news/tyrese-haliburton-among-players-in-danger-of-missing-all-nba-teams-awards-because-of-leagues-65-game-rule/#:~:text=Perhaps%20the%20most%20notable%20aspect,MVP
stats["Eligibility_65G"] = (stats["G"] >= 65).astype(int)
stats_2024["Eligibility_65G"] = (stats_2024["G"] >= 65).astype(int)

In [239]:
predictors =  ['Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
        '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year',
       'W', 'L', 'W/L%', 'GB', 'PS/G','PA/G', 'SRS', 'Eligibility_65G']

In [240]:
# We are now going to split the data into training and test data
train = stats[stats["Year"] < 2023]
test = stats[stats["Year"] == 2023]

In [241]:
stats2 = stats[:]
stat_ratios = stats2[["PTS", "AST", "STL", "BLK", "3P", "Year"]].groupby("Year").apply(lambda x: x/x.mean())
stat_ratios.reset_index(drop=True, inplace=True)
stats2.reset_index(drop=True, inplace=True)
stats2[["PTS_R", "AST_R", "STL_R", "BLK_R", "3P_R"]] = stat_ratios[["PTS", "AST", "STL", "BLK", "3P"]]
predictors2 = predictors + ["PTS_R", "AST_R", "STL_R", "BLK_R", "3P_R"]

/var/folders/7y/x2g2fgds6c9d9xrjrvscg14r0000gn/T/ipykernel_64808/83561794.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  stat_ratios = stats2[["PTS", "AST", "STL", "BLK", "3P", "Year"]].groupby("Year").apply(lambda x: x/x.mean())
/var/folders/7y/x2g2fgds6c9d9xrjrvscg14r0000gn/T/ipykernel_64808/83561794.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stats2[["PTS_R", "AST_R", "STL_R", "BLK_R", "3P_R"]] = stat_ratios[["PTS", "AST", "STL", "BLK", "3P"

In [242]:
stats_2024_2 = stats_2024[:]
stat_ratios_2024 = stats_2024_2[["PTS", "AST", "STL", "BLK", "3P", "Year"]].groupby("Year").apply(lambda x: x/x.mean())
stat_ratios_2024.reset_index(drop=True, inplace=True)
stats_2024_2.reset_index(drop=True, inplace=True)
stats_2024_2[["PTS_R", "AST_R", "STL_R", "BLK_R", "3P_R"]] = stat_ratios_2024[["PTS", "AST", "STL", "BLK", "3P"]]

/var/folders/7y/x2g2fgds6c9d9xrjrvscg14r0000gn/T/ipykernel_64808/1787446280.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  stat_ratios_2024 = stats_2024_2[["PTS", "AST", "STL", "BLK", "3P", "Year"]].groupby("Year").apply(lambda x: x/x.mean())
/var/folders/7y/x2g2fgds6c9d9xrjrvscg14r0000gn/T/ipykernel_64808/1787446280.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stats_2024_2[["PTS_R", "AST_R", "STL_R", "BLK_R", "3P_R"]] = stat_ratios_2024[["PTS",

In [243]:
train2 = stats2[stats2["Year"] < 2023]
test2 = stats2[stats2["Year"] == 2023]

In [244]:
from sklearn.linear_model import Ridge

reg = Ridge(alpha= 1.0)
reg2 = Ridge(alpha=1.0)

In [245]:
reg.fit(train[predictors], train["Share"])
reg2.fit(train2[predictors2], train2["Share"])

Ridge()

In [246]:
# Using Average precision for error metric (if I predicted correctly from the top 15, then I get a 1, and if i didn't it will give me a score based on how far off the rpediction was)
def find_ap(combination):
    actual = combination.sort_values("Share", ascending=False).head(5)
    predicted = combination.sort_values("predictions", ascending=False)
    ps = []
    found = 0
    seen = 1
    for index, row in predicted.iterrows():
        if row["Player"] in actual["Player"].values:
            found += 1
            ps.append(found/seen)
        seen +=1
    return sum(ps) / len(ps)

In [247]:
def add_ranks(predictions):
    predictions = predictions.sort_values("predictions", ascending=False)
    predictions["Predicted_Rk"] = list(range(1,predictions.shape[0]+1))
    predictions = predictions.sort_values("Share", ascending=False)
    predictions["Rk"] = list(range(1,predictions.shape[0]+1))
    predictions["Diff"] = (predictions["Rk"] - predictions["Predicted_Rk"])
    return predictions

In [249]:
# this backtest function just makes it so that we have a more concrete way of testing the data, it includes the use of the other functions that we wrote that makes it easier to easity use this 
from sklearn.metrics import r2_score
def backtest(stats, model, years, predictors):
    aps = []
    r2_scores = []
    all_predictions = []

    for year in years:
        train = stats[stats["Year"] < year]
        test = stats[stats["Year"] == year]

        model.fit(train[predictors], train["Share"])
        predictions = model.predict(test[predictors])
        predictions_df = pd.DataFrame(predictions, columns=["predictions"], index=test.index)

        combination = pd.concat([test[["Player", "Share"]], predictions_df], axis=1)
        combination = add_ranks(combination)
        all_predictions.append(combination)

        ap = find_ap(combination)
        aps.append(ap)

        r2 = r2_score(test["Share"], predictions)
        r2_scores.append(r2)

    mean_ap = sum(aps) / len(aps)
    mean_r2 = sum(r2_scores) / len(r2_scores)

    return mean_ap, mean_r2, aps, pd.concat(all_predictions)


In [250]:
years = list(range(2003, 2024))

In [251]:
Model_error_MAP = {}
Model_error_r2 = {}
Model_error_MAP_regular_model = {}
Model_error_r2_regular_model = {}

In [252]:
from sklearn.linear_model import Ridge

# Ridge Regression 
alpha_values = [0.001, 0.01, 0.1, 1, 10]
results1 = {}
results2 = {}
for alpha in alpha_values:
    reg = Ridge(alpha=alpha, random_state=10)
    mean_ap, mean_r2, aps, all_predictions = backtest(stats, reg, years[5:], predictors)
    results1[alpha] = mean_ap
    results2[alpha] = mean_r2

best_alpha1 = max(results1, key=results1.get)
best_alpha2 = max(results2, key=results2.get)

reg1_regular = Ridge(alpha=best_alpha1, random_state=10)
mean_ap_1, mean_r2_1, aps, all_predictions = backtest(stats, reg1_regular, years[5:], predictors)

reg2_regular = Ridge(alpha=best_alpha2, random_state=10)
mean_ap_2, mean_r2_2, aps, all_predictions = backtest(stats, reg2_regular, years[5:], predictors)

Model_error_MAP_regular_model["Ridge"] = mean_ap_1
Model_error_r2_regular_model["Ridge"] = mean_r2_2


In [253]:
# RIDGE REGRESSION WITH FEATURED ENGINEERED VARIABLES
alpha_values = [0.001, 0.01, 0.1, 1, 10]
results1 = {}
results2 = {}
for alpha in alpha_values:
    reg = Ridge(alpha=alpha, random_state=10)
    mean_ap, mean_r2, aps, all_predictions = backtest(stats2, reg, years[5:], predictors2)
    results1[alpha] = mean_ap
    results2[alpha] = mean_r2

best_alpha1 = max(results1, key=results1.get)
best_alpha2 = max(results2, key=results2.get)

reg1 = Ridge(alpha=best_alpha1, random_state=10)
mean_ap_1, mean_r2_1, aps, all_predictions = backtest(stats2, reg1, years[5:], predictors2)

reg2 = Ridge(alpha=best_alpha2, random_state=10)
mean_ap_2, mean_r2_2, aps, all_predictions = backtest(stats2, reg2, years[5:], predictors2)

Model_error_MAP["Ridge"] = mean_ap_1
Model_error_r2["Ridge"] = mean_r2_2

In [254]:
# prediction using ridge repgression with alpha for best MAP!
over_65_games_2024 = stats_2024[stats_2024["G"] >= 65]
predictions_2024 = reg1_regular.predict(over_65_games_2024[predictors])
predictions_2024 = pd.DataFrame(predictions_2024, columns=["predictions"], index=over_65_games_2024.index)
combination_2024 = pd.concat([over_65_games_2024[["Player"]], predictions_2024], axis=1)
combination_2024.sort_values("predictions", ascending=False).head(15)

,Player,predictions
314,Giannis Antetokounmpo,0.530208
124,Luka Dončić,0.504942
398,Shai Gilgeous-Alexander,0.433715
143,Nikola Jokić,0.430467
242,Anthony Davis,0.346503
254,LeBron James,0.338902
24,Jayson Tatum,0.312839
511,Victor Wembanyama,0.289426
449,Kevin Durant,0.273426
370,Jalen Brunson,0.265188


In [255]:
# prediction using ridge repgression with alpha for best r2 score!
over_65_games_2024 = stats_2024[stats_2024["G"] >= 65]
predictions_2024 = reg2_regular.predict(over_65_games_2024[predictors])
predictions_2024 = pd.DataFrame(predictions_2024, columns=["predictions"], index=over_65_games_2024.index)
combination_2024 = pd.concat([over_65_games_2024[["Player"]], predictions_2024], axis=1)
combination_2024.sort_values("predictions", ascending=False).head(15)

,Player,predictions
314,Giannis Antetokounmpo,0.532928
124,Luka Dončić,0.500142
398,Shai Gilgeous-Alexander,0.436067
143,Nikola Jokić,0.431820
242,Anthony Davis,0.348253
254,LeBron James,0.337525
24,Jayson Tatum,0.309013
511,Victor Wembanyama,0.289052
449,Kevin Durant,0.271428
370,Jalen Brunson,0.264517


In [256]:
# prediction using ridge repgression WITH FEATURED VARIABLES for best alpha value for map score!
over_65_games_2024 = stats_2024_2[stats_2024_2["G"] >= 65]
predictions_2024 = reg1.predict(over_65_games_2024[predictors2])
predictions_2024 = pd.DataFrame(predictions_2024, columns=["predictions"], index=over_65_games_2024.index)
combination_2024 = pd.concat([over_65_games_2024[["Player"]], predictions_2024], axis=1)
combination_2024.sort_values("predictions", ascending=False).head(15)

,Player,predictions
314,Giannis Antetokounmpo,0.529144
124,Luka Dončić,0.513612
143,Nikola Jokić,0.432466
398,Shai Gilgeous-Alexander,0.425761
242,Anthony Davis,0.344480
254,LeBron James,0.343009
24,Jayson Tatum,0.317870
511,Victor Wembanyama,0.288445
449,Kevin Durant,0.278539
370,Jalen Brunson,0.268511


In [257]:
# prediction using ridge repgression WITH FEATURED VARIABLES for best alpha value for r2 score!
over_65_games_2024 = stats_2024_2[stats_2024_2["G"] >= 65]
predictions_2024 = reg2.predict(over_65_games_2024[predictors2])
predictions_2024 = pd.DataFrame(predictions_2024, columns=["predictions"], index=over_65_games_2024.index)
combination_2024 = pd.concat([over_65_games_2024[["Player"]], predictions_2024], axis=1)
combination_2024.sort_values("predictions", ascending=False).head(15)

,Player,predictions
314,Giannis Antetokounmpo,0.537654
124,Luka Dončić,0.503360
143,Nikola Jokić,0.435885
398,Shai Gilgeous-Alexander,0.433355
242,Anthony Davis,0.349321
254,LeBron James,0.340357
24,Jayson Tatum,0.310446
511,Victor Wembanyama,0.288565
449,Kevin Durant,0.274599
370,Jalen Brunson,0.269041


In [287]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=50, random_state=1, min_samples_split=5)
mean_ap, mean_r2, aps, all_predictions = backtest(stats, rf, years[5:], predictors)
Model_error_MAP_regular_model["Random Forest"] = mean_ap
Model_error_r2_regular_model["Random Forest"] = mean_r2

In [288]:
rf2 = RandomForestRegressor(n_estimators=50, random_state=1, min_samples_split=5)
mean_ap, mean_r2, aps, all_predictions = backtest(stats2, rf2, years[5:], predictors2)
Model_error_MAP["Random Forest"] = mean_ap
Model_error_r2["Random Forest"] = mean_r2

In [289]:
#Prediction using RandomForest Regressor:
over_65_games_2024 = stats_2024[stats_2024["G"] >= 65]
predictions_2024 = rf.predict(over_65_games_2024[predictors])
predictions_2024 = pd.DataFrame(predictions_2024, columns=["predictions"], index=over_65_games_2024.index)
combination_2024 = pd.concat([over_65_games_2024[["Player"]], predictions_2024], axis=1)
combination_2024.sort_values("predictions", ascending=False).head(15)

,Player,predictions
398,Shai Gilgeous-Alexander,0.927564
124,Luka Dončić,0.874434
24,Jayson Tatum,0.801648
314,Giannis Antetokounmpo,0.788101
143,Nikola Jokić,0.749174
326,Anthony Edwards,0.732508
254,LeBron James,0.724161
449,Kevin Durant,0.601082
242,Anthony Davis,0.542435
23,Jaylen Brown,0.478845


In [290]:
#Prediction using RandomForest Regressor with engineered features:
over_65_games_2024_2 = stats_2024_2[stats_2024_2["G"] >= 65]
predictions_2024 = rf2.predict(over_65_games_2024_2[predictors2])
predictions_2024 = pd.DataFrame(predictions_2024, columns=["predictions"], index=over_65_games_2024_2.index)
combination_2024 = pd.concat([over_65_games_2024_2[["Player"]], predictions_2024], axis=1)
combination_2024.sort_values("predictions", ascending=False).head(15)

,Player,predictions
124,Luka Dončić,0.808195
314,Giannis Antetokounmpo,0.799152
24,Jayson Tatum,0.792201
398,Shai Gilgeous-Alexander,0.791987
254,LeBron James,0.737688
143,Nikola Jokić,0.730206
326,Anthony Edwards,0.706824
449,Kevin Durant,0.640606
242,Anthony Davis,0.583055
23,Jaylen Brown,0.482164


In [291]:
import xgboost as xgb
xgb_model = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, use_label_encoder=False, eval_metric='map', random_state=1)
mean_ap, mean_r2, aps, all_predictions = backtest(stats, xgb_model, years[5:], predictors)
Model_error_MAP_regular_model["XGBoost"] = mean_ap
Model_error_r2_regular_model["XGBoost"] = mean_r2

In [292]:
xgb_model2 = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, use_label_encoder=False, eval_metric='map', random_state=1)
mean_ap, mean_r2, aps, all_predictions = backtest(stats2, xgb_model2, years[5:], predictors2)
Model_error_MAP["XGBoost"] = mean_ap
Model_error_r2["XGBoost"] = mean_r2

In [264]:
#Prediction using XGBoost Regressor
over_65_games_2024 = stats_2024[stats_2024["G"] >= 65]
predictions_2024 = xgb_model.predict(over_65_games_2024[predictors])
predictions_2024 = pd.DataFrame(predictions_2024, columns=["predictions"], index=over_65_games_2024.index)
combination_2024 = pd.concat([over_65_games_2024[["Player"]], predictions_2024], axis=1)
combination_2024.sort_values("predictions", ascending=False).head(15)

,Player,predictions
143,Nikola Jokić,1.001049
398,Shai Gilgeous-Alexander,0.890962
314,Giannis Antetokounmpo,0.886532
24,Jayson Tatum,0.814714
124,Luka Dončić,0.797981
449,Kevin Durant,0.682752
326,Anthony Edwards,0.658857
254,LeBron James,0.575331
242,Anthony Davis,0.537727
370,Jalen Brunson,0.487120


In [265]:
#Prediction using XGBoost Regressor with engineered variables:
over_65_games_2024_2 = stats_2024_2[stats_2024_2["G"] >= 65]
predictions_2024 = xgb_model2.predict(over_65_games_2024_2[predictors2])
predictions_2024 = pd.DataFrame(predictions_2024, columns=["predictions"], index=over_65_games_2024_2.index)
combination_2024 = pd.concat([over_65_games_2024_2[["Player"]], predictions_2024], axis=1)
combination_2024.sort_values("predictions", ascending=False).head(15)

,Player,predictions
143,Nikola Jokić,0.943112
398,Shai Gilgeous-Alexander,0.932347
124,Luka Dončić,0.844286
314,Giannis Antetokounmpo,0.839916
24,Jayson Tatum,0.826600
326,Anthony Edwards,0.665400
449,Kevin Durant,0.656493
254,LeBron James,0.605061
242,Anthony Davis,0.510394
370,Jalen Brunson,0.501441


In [272]:
# Lasso REGRESSION
from sklearn.linear_model import Lasso
 
alpha_values = [0.001, 0.01, 0.1, 1, 10]
results1 = {}
results2 = {}
for alpha in alpha_values:
    reg = Lasso(alpha=alpha, random_state=10)
    mean_ap, mean_r2, aps, all_predictions = backtest(stats, reg, years[5:], predictors)
    results1[alpha] = mean_ap
    results2[alpha] = mean_r2

best_alpha1 = max(results1, key=results1.get)
best_alpha2 = max(results2, key=results2.get)

leg1_regular = Lasso(alpha=best_alpha1, random_state=10)
mean_ap_1, mean_r2_1, aps, all_predictions = backtest(stats, leg1_regular, years[5:], predictors)

leg2_regular = Lasso(alpha=best_alpha2, random_state=10)
mean_ap_2, mean_r2_2, aps, all_predictions = backtest(stats, leg2_regular, years[5:], predictors)

Model_error_MAP_regular_model["Lasso"] = mean_ap_1
Model_error_r2_regular_model["Lasso"] = mean_r2_2

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.815e-01, tolerance: 2.939e-03
  model = cd_fast.enet_coordinate_descent(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.364e-01, tolerance: 3.536e-03
  model = cd_fast.enet_coordinate_descent(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iter

In [273]:
from sklearn.linear_model import Lasso
 # RIDGE REGRESSION WITH FEATURED ENGINEERED VARIABLES
alpha_values = [0.001, 0.01, 0.1, 1, 10]
results1 = {}
results2 = {}
for alpha in alpha_values:
    reg = Lasso(alpha=alpha, random_state=10)
    mean_ap, mean_r2, aps, all_predictions = backtest(stats, reg, years[5:], predictors)
    results1[alpha] = mean_ap
    results2[alpha] = mean_r2

best_alpha1 = max(results1, key=results1.get)
best_alpha2 = max(results2, key=results2.get)

leg1 = Lasso(alpha=best_alpha1, random_state=10)
mean_ap_1, mean_r2_1, aps, all_predictions = backtest(stats2, leg1, years[5:], predictors2)

leg2 = Lasso(alpha=best_alpha2, random_state=10)
mean_ap_2, mean_r2_2, aps, all_predictions = backtest(stats2, leg2, years[5:], predictors2)

Model_error_MAP["Lasso"] = mean_ap_1
Model_error_r2["Lasso"] = mean_r2_2

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.815e-01, tolerance: 2.939e-03
  model = cd_fast.enet_coordinate_descent(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.364e-01, tolerance: 3.536e-03
  model = cd_fast.enet_coordinate_descent(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iter

In [274]:
#Prediction using Lasso Regression for alpha with best MAP:
over_65_games_2024 = stats_2024[stats_2024["G"] >= 65]
predictions_2024 = leg1_regular.predict(over_65_games_2024[predictors])
predictions_2024 = pd.DataFrame(predictions_2024, columns=["predictions"], index=over_65_games_2024.index)
combination_2024 = pd.concat([over_65_games_2024[["Player"]], predictions_2024], axis=1)
combination_2024.sort_values("predictions", ascending=False).head(15)

,Player,predictions
124,Luka Dončić,0.479378
314,Giannis Antetokounmpo,0.444548
143,Nikola Jokić,0.375381
398,Shai Gilgeous-Alexander,0.365587
254,LeBron James,0.307304
24,Jayson Tatum,0.305106
242,Anthony Davis,0.292227
511,Victor Wembanyama,0.262990
370,Jalen Brunson,0.254778
449,Kevin Durant,0.250864


In [275]:
#Prediction using Lasso Regression for alpha with best r2 score:
over_65_games_2024 = stats_2024[stats_2024["G"] >= 65]
predictions_2024 = leg2_regular.predict(over_65_games_2024[predictors])
predictions_2024 = pd.DataFrame(predictions_2024, columns=["predictions"], index=over_65_games_2024.index)
combination_2024 = pd.concat([over_65_games_2024[["Player"]], predictions_2024], axis=1)
combination_2024.sort_values("predictions", ascending=False).head(15)

,Player,predictions
124,Luka Dončić,0.479378
314,Giannis Antetokounmpo,0.444548
143,Nikola Jokić,0.375381
398,Shai Gilgeous-Alexander,0.365587
254,LeBron James,0.307304
24,Jayson Tatum,0.305106
242,Anthony Davis,0.292227
511,Victor Wembanyama,0.262990
370,Jalen Brunson,0.254778
449,Kevin Durant,0.250864


In [276]:
#Prediction using Lasso with engineered variables with best MAP:
over_65_games_2024_2 = stats_2024_2[stats_2024_2["G"] >= 65]
predictions_2024 = leg1.predict(over_65_games_2024_2[predictors2])
predictions_2024 = pd.DataFrame(predictions_2024, columns=["predictions"], index=over_65_games_2024_2.index)
combination_2024 = pd.concat([over_65_games_2024_2[["Player"]], predictions_2024], axis=1)
combination_2024.sort_values("predictions", ascending=False).head(15)

,Player,predictions
124,Luka Dončić,0.478934
314,Giannis Antetokounmpo,0.444618
143,Nikola Jokić,0.375359
398,Shai Gilgeous-Alexander,0.365541
254,LeBron James,0.307146
24,Jayson Tatum,0.304807
242,Anthony Davis,0.292307
511,Victor Wembanyama,0.262882
370,Jalen Brunson,0.254538
449,Kevin Durant,0.250693


In [277]:
#Prediction using Lasso with engineered variables with best r2:
over_65_games_2024_2 = stats_2024_2[stats_2024_2["G"] >= 65]
predictions_2024 = leg2.predict(over_65_games_2024_2[predictors2])
predictions_2024 = pd.DataFrame(predictions_2024, columns=["predictions"], index=over_65_games_2024_2.index)
combination_2024 = pd.concat([over_65_games_2024_2[["Player"]], predictions_2024], axis=1)
combination_2024.sort_values("predictions", ascending=False).head(15)

,Player,predictions
124,Luka Dončić,0.478934
314,Giannis Antetokounmpo,0.444618
143,Nikola Jokić,0.375359
398,Shai Gilgeous-Alexander,0.365541
254,LeBron James,0.307146
24,Jayson Tatum,0.304807
242,Anthony Davis,0.292307
511,Victor Wembanyama,0.262882
370,Jalen Brunson,0.254538
449,Kevin Durant,0.250693


In [278]:
from sklearn.linear_model import LinearRegression
linear_model = LinearRegression()
mean_ap, mean_r2, aps, all_predictions = backtest(stats, linear_model, years[5:], predictors)
Model_error_MAP_regular_model["Linear"] = mean_ap_1
Model_error_r2_regular_model["Linear"] = mean_r2_2

In [279]:
linear_model2 = LinearRegression()
mean_ap, mean_r2, aps, all_predictions = backtest(stats2, linear_model2, years[5:], predictors2)
Model_error_MAP["Linear"] = mean_ap_1
Model_error_r2["Linear"] = mean_r2_2

In [280]:
#Prediction using Linear Regression:
over_65_games_2024 = stats_2024[stats_2024["G"] >= 65]
predictions_2024 = linear_model.predict(over_65_games_2024[predictors])
predictions_2024 = pd.DataFrame(predictions_2024, columns=["predictions"], index=over_65_games_2024.index)
combination_2024 = pd.concat([over_65_games_2024[["Player"]], predictions_2024], axis=1)
combination_2024.sort_values("predictions", ascending=False).head(15)

,Player,predictions
314,Giannis Antetokounmpo,0.532970
124,Luka Dončić,0.500063
398,Shai Gilgeous-Alexander,0.436103
143,Nikola Jokić,0.431842
242,Anthony Davis,0.348281
254,LeBron James,0.337502
24,Jayson Tatum,0.308949
511,Victor Wembanyama,0.289043
449,Kevin Durant,0.271395
370,Jalen Brunson,0.264502


In [281]:
#Prediction using Linear Regression with engineered variables:
over_65_games_2024_2 = stats_2024_2[stats_2024_2["G"] >= 65]
predictions_2024 = linear_model2.predict(over_65_games_2024_2[predictors2])
predictions_2024 = pd.DataFrame(predictions_2024, columns=["predictions"], index=over_65_games_2024_2.index)
combination_2024 = pd.concat([over_65_games_2024_2[["Player"]], predictions_2024], axis=1)
combination_2024.sort_values("predictions", ascending=False).head(15)

,Player,predictions
314,Giannis Antetokounmpo,0.537694
124,Luka Dončić,0.503279
143,Nikola Jokić,0.435904
398,Shai Gilgeous-Alexander,0.433389
242,Anthony Davis,0.349346
254,LeBron James,0.340331
24,Jayson Tatum,0.310382
511,Victor Wembanyama,0.288550
449,Kevin Durant,0.274564
370,Jalen Brunson,0.269026


In [296]:
for model, error in Model_error_MAP_regular_model.items():
    print(f"For {model} the MAP is {error}")


For Ridge the MAP is 0.8115069159641528
For Random Forest the MAP is 0.7706763491138491
For XGBoost the MAP is 0.7758780425227794
For SVR the MAP is 0.3738365325034506
For Lasso the MAP is 0.7781491251803752
For Linear the MAP is 0.7737741251803751


In [297]:
for model, error in Model_error_r2_regular_model.items():
    print(f"For {model} the r2 score is {error}")

For Ridge the r2 score is 0.38407548283604087
For XGBoost the r2 score is 0.7061098144023251
For Lasso the r2 score is 0.3744512276926478
For Linear the r2 score is 0.3743569180834421
For Random Forest the r2 score is 0.7005377033824208
For SVR the r2 score is -0.7388758211379292


In [300]:
for model, error in Model_error_MAP.items():
    print(f"For {model} the MAP score is {error}")

For Ridge the MAP score is 0.8154732461166285
For Random Forest the MAP score is 0.7492353871973437
For XGBoost the MAP score is 0.7563971792096792
For SVR the MAP score is 0.3619768893245145
For Lasso the MAP score is 0.7737741251803751
For Linear the MAP score is 0.7737741251803751


In [299]:
for model, error in Model_error_r2.items():
    print(f"For {model} the r2 score is {error}")

For Ridge the r2 score is 0.3837695243083318
For XGBoost the r2 score is 0.6995880934991986
For Lasso the r2 score is 0.3743569180834421
For Linear the r2 score is 0.3743569180834421
For Random Forest the r2 score is 0.694635719545249
For SVR the r2 score is -0.7388292151170361
